In [5]:
# %pip install python-dotenv
# %pip install openai
# %pip install "pinecone-client[grpc]"
# %pip install -q -U google-generativeai

In [1]:
from dotenv import load_dotenv
load_dotenv()
import os
from pinecone import Pinecone, ServerlessSpec

C:\Users\saad2\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [12]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

In [13]:
pc.create_index(
    name="rag",
    dimension=768,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

In [14]:
import json

data = json.load(open("reviews.json"))
display(data["reviews"])

[{'professor': 'Dr. Alice Johnson',
  'subject': 'Calculus',
  'stars': 5,
  'review': 'Dr. Johnson explains concepts very clearly and makes complex topics easy to understand.'},
 {'professor': 'Dr. Michael Smith',
  'subject': 'Linear Algebra',
  'stars': 4,
  'review': 'Great professor, but sometimes moves too fast through the material.'},
 {'professor': 'Dr. Emily Davis',
  'subject': 'Statistics',
  'stars': 3,
  'review': 'Lectures are okay, but the exams are much harder than the examples in class.'},
 {'professor': 'Dr. Robert Brown',
  'subject': 'Differential Equations',
  'stars': 2,
  'review': 'Dr. Brown is knowledgeable, but his teaching style is hard to follow.'},
 {'professor': 'Dr. Linda White',
  'subject': 'Discrete Mathematics',
  'stars': 5,
  'review': 'Excellent professor! Makes the class very engaging and fun.'},
 {'professor': 'Dr. William Taylor',
  'subject': 'Abstract Algebra',
  'stars': 3,
  'review': 'The course content is challenging, but Dr. Taylor is hel

In [6]:
# from openai import OpenAI

# process_data = []
# client = OpenAI()

# # Creating embeddings
# # embedding capture the semantic representation of the text in numerical form
# # man and uncle would be more semantically related than man and woman

# for review in data["reviews"]:
#     response = client.embeddings.create(
#         input=review["review"],
#         model="text-embedding-3-small"
#     )
#     embedding = response.data[0].embedding
#     process_data.append({
#         "values": embedding,
#         "id": review["professor"],
#         "metadata": {
#             "review": review["review"],
#             "subject": review["subject"],
#             "stars": review["stars"]
#         } 
#     })

In [15]:
import google.generativeai as genai

genai.configure(api_key=os.environ["GEMINI_API_KEY"])

In [8]:
# model = genai.GenerativeModel("gemini-1.5-flash")
# response = model.generate_content("Write a story about a magic backpack.")
# print(response.text)

Amelia was never one for ordinary things. She dreamt of adventure, of faraway lands and mythical creatures. But life in the sleepy town of Willow Creek offered little excitement beyond the annual pumpkin pie contest. That is, until she found the backpack.

It wasn't much to look at, a faded blue canvas with worn leather straps. But when Amelia pulled the zipper, the world inside shimmered. Golden light spilled out, revealing a miniature landscape – a sun-drenched beach with palm trees swaying in the breeze and a sparkling ocean lapping at the shore.

Amelia gasped, reaching out a hand. The sand felt warm beneath her fingers, and the salty air filled her lungs. The backpack, she realized, was a portal to anywhere she wished. With a thrill of excitement, she pulled out a map from her own bag and chose a location: the Amazon Rainforest.

The next moment, she was standing in a jungle so dense it felt like a living, breathing creature. Monkeys swung from the vines, macaws screeched overhead

In [19]:
process_data = []

# # Creating embeddings
# # embedding capture the semantic representation of the text in numerical form
# # man and uncle would be more semantically related than man and woman

for review in data["reviews"]:
    response = genai.embed_content(
        model="models/text-embedding-004",
        content=review["review"]
    )

    embedding = response["embedding"]

    process_data.append({
        "values": embedding,
        "id": review["professor"],
        "metadata": {
            "review": review["review"],
            "subject": review["subject"],
            "stars": review["stars"]
        } 
    })


In [20]:
process_data[0]

{'values': [0.029872103,
  0.009945141,
  -0.028079951,
  -0.0009476907,
  0.009116488,
  0.022909852,
  -0.008742504,
  -0.001438194,
  -0.05563486,
  0.025488919,
  0.05368564,
  0.009556996,
  0.015154428,
  0.0005151121,
  -0.002315483,
  -0.08070704,
  -0.00014856545,
  0.024148127,
  -0.105188265,
  0.0152481,
  0.016392397,
  -0.01666546,
  0.017610403,
  -0.015748577,
  0.023324661,
  0.019071246,
  0.0036823105,
  -0.045127727,
  0.0582674,
  -0.042742755,
  0.04362896,
  -0.0033807897,
  0.005089445,
  -0.030479945,
  -0.06361912,
  0.013693242,
  -0.018871076,
  -0.019639393,
  0.057489067,
  -0.011918998,
  -0.023496732,
  -0.022098873,
  -0.029326709,
  -0.0050288606,
  -0.03064818,
  0.013084556,
  -0.011982349,
  0.092815295,
  0.033713892,
  0.07761495,
  0.04852356,
  0.049334385,
  -0.08434088,
  0.07528663,
  -0.0025415826,
  -0.025968261,
  -0.050893363,
  -0.024849882,
  0.039758615,
  -0.027117493,
  -0.03228314,
  -0.011419421,
  -0.008501899,
  -0.05627641,
  0.

In [21]:
len(process_data)

20

In [22]:
index = pc.Index("rag")
index.upsert(
    vectors=process_data,
    namespace="ns1"
)

{'upserted_count': 20}

In [23]:
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 20}},
 'total_vector_count': 20}

In [32]:
# %pip install pip-chill
# %pip pip-chill >requirements.txt